In [1]:
##imports 

import cv2 
import pytesseract
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import confusion_matrix

In [2]:
##OCR
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Adding custom options
custom_config = r'--oem 3 --psm 6'

In [3]:
##testing purposes
trainingData = pd.read_csv("train-data.csv") 
testingData  = pd.read_csv("test.csv") 

In [ ]:
## PREPROCESSING - BEGIN
trainingDataTBA = trainingData


In [ ]:
trainingDataTBA.head(10)    

In [ ]:
trainingDataTBA['OCR text']=trainingDataTBA['OCR text'].str.replace('\n'," ")
trainingDataTBA['OCR text']=trainingDataTBA['OCR text'].str.replace('.'," ")
trainingDataTBA['OCR text']=trainingDataTBA['OCR text'].str.replace(','," ")
trainingDataTBA['OCR text']=trainingDataTBA['OCR text'].str.replace('@'," ")
trainingDataTBA['OCR text']=trainingDataTBA['OCR text'].str.replace(';'," ")
trainingDataTBA['OCR text']=trainingDataTBA['OCR text'].str.replace('/'," ")
trainingDataTBA['OCR text']=trainingDataTBA['OCR text'].str.replace(''," ")

In [ ]:
trainingData = trainingDataTBA

In [ ]:
trainingData.head(10)    

## PREPROCESSING - END


In [4]:
##batch size (started with a small no. to train the algorithm on)
test_var = 320000

In [5]:
##vectorizing the data
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(trainingData[:test_var]['OCR text'])

In [10]:
y = trainingData[:test_var]["Label"]

In [10]:
print(X)


  (0, 1023804)	0.17356085231708732
  (0, 3902906)	0.11006352215199676
  (0, 1994521)	0.17368667202763452
  (0, 1105205)	0.10576162854433116
  (0, 486789)	0.2621773100694327
  (0, 613354)	0.2228816039914777
  (0, 615186)	0.20749094914032148
  (0, 3269928)	0.12909278246115696
  (0, 1565801)	0.10914941928698993
  (0, 2774797)	0.0990143902998192
  (0, 3522610)	0.10883672516232283
  (0, 2869639)	0.09994235132014762
  (0, 3269883)	0.22967632373744243
  (0, 1154145)	0.1358293341451583
  (0, 640366)	0.16160764306247738
  (0, 3387834)	0.07002938197734061
  (0, 1271309)	0.08842233560437013
  (0, 1111183)	0.11582977767520121
  (0, 3280571)	0.24496235065522273
  (0, 346899)	0.2119889565398721
  (0, 3387681)	0.08155898259578934
  (0, 2918596)	0.21840229149025575
  (0, 785441)	0.2460772720436303
  (0, 362384)	0.22574311148663473
  (0, 2169322)	0.2748617352756527
  :	:
  (319999, 1347754)	0.3541882361429565
  (319999, 2597553)	0.3541882361429565
  (319999, 378591)	0.2831840859748654
  (319999, 230298

In [ ]:
print(vectorizer.get_feature_names())

In [11]:
print(X.shape)

(320000, 3963685)


In [20]:
##Creating pipelines for SVM

from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
mnb_pipeline = Pipeline([
    ('svm', SGDClassifier(max_iter=320000))
])

In [ ]:
##Listing the testing tuning parameters

import numpy as np
from sklearn.model_selection import GridSearchCV
grid_params = {    
    "svm__loss" : ["hinge", "log", "squared_hinge", "modified_huber"],
    "svm__alpha" : [0.0001, 0.001, 0.01, 0.1],
    "svm__penalty" : ["l2", "l1", "none"],
}
clf = GridSearchCV(mnb_pipeline, grid_params,verbose=10)
clf.fit(X, y)
print("Best Score: ", clf.best_score_)
print("Best Params: ", clf.best_params_)


Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] svm__alpha=0.0001, svm__loss=hinge, svm__penalty=l2 .............
[CV]  svm__alpha=0.0001, svm__loss=hinge, svm__penalty=l2, score=0.722, total=  31.2s
[CV] svm__alpha=0.0001, svm__loss=hinge, svm__penalty=l2 .............
[CV]  svm__alpha=0.0001, svm__loss=hinge, svm__penalty=l2, score=0.724, total=  33.3s
[CV] svm__alpha=0.0001, svm__loss=hinge, svm__penalty=l2 .............
[CV]  svm__alpha=0.0001, svm__loss=hinge, svm__penalty=l2, score=0.725, total=  31.6s
[CV] svm__alpha=0.0001, svm__loss=hinge, svm__penalty=l2 .............
[CV]  svm__alpha=0.0001, svm__loss=hinge, svm__penalty=l2, score=0.725, total=  32.2s
[CV] svm__alpha=0.0001, svm__loss=hinge, svm__penalty=l2 .............
[CV]  svm__alpha=0.0001, svm__loss=hinge, svm__penalty=l2, score=0.726, total=  30.1s
[CV] svm__alpha=0.0001, svm__loss=hinge, svm__penalty=l1 .............
[CV]  svm__alpha=0.0001, svm__loss=hinge, svm__penalty=l1, score=0.560, total= 1.1

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   31.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.1min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.6min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  2.1min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  2.6min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  3.8min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:  4.9min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  6.0min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  7.2min remaining:    0.0s


In [13]:
##Fiting the data and training it - modify tuning parameters here based on the pipelines
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(alpha=0.0001, loss='hinge', penalty='none')

clf.fit(X, y)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='none',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [14]:
##Testing data based on the fitted model

from tqdm import tqdm

#Prediction and accuracy
from sklearn.metrics import accuracy_score
documents1 = testingData[:]["OCR text"]
y_true = testingData[:]['Label']
input = vectorizer.transform(documents1)
prediction1= clf.predict(input)

dataframe = pd.DataFrame(list(y_true),prediction1)
index = 0
count = 0
for i in tqdm(list(y_true)):
    if i == prediction1[index]:
        count += 1
    index+=1
    
    
print("Accuracy:",accuracy_score(list(y_true),prediction1))
print(dataframe)

100%|██████████| 40000/40000 [00:00<00:00, 1110044.73it/s]


0.752525
     0
11  11
0    0
0    0
9    4
14  14
..  ..
14  10
11  11
12  15
9    9
15  15

[40000 rows x 1 columns]
